In [ ]:
import pandas as pd
from nilearn import image, plotting, input_data, glm
#from nilearn.glm import threshold_stats_img
import numpy as np

from nilearn.input_data import NiftiMasker
import nibabel as nib


import os
import statsmodels.api as sm
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template
import matplotlib.pyplot as plt
import pdb
from scipy.stats import gamma
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#for each sub overlap two cluster masks (and for each individual mask)
#set a 1 everywhere they overlap
#add it to a global image so you get a count of how many overlaps there are for that conjunction
#do binomial on each voxel
#make new mask
